In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel
import torch

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generate retrieval corpus

In [2]:
path = '/Users/adamwagnerhoegh/Documents/Legal data/domsdatabasen.retsinformation_newer.json'

with open(path) as f:
    retsinfo = json.load(f)

rag_list = []
idx = 0
for lov in tqdm(retsinfo):
    for kapitel in lov['kapitler']:
        lov_navn = lov['shortName']
        for paragraffer in kapitel['paragraffer']:
            temp_paragraf_dict = {}
            temp_paragraf_dict['paragraf_nr'] = paragraffer['nummer']
            temp_paragraf_dict['lovnavn'] = lov_navn
            temp_paragraf_list = []
            for styk in paragraffer['stk']:
                temp_paragraf_list.append(styk['tekst'])
            temp_paragraf_dict['text'] = ' '.join(temp_paragraf_list)
            rag_list.append(temp_paragraf_dict)

with open("rag_list.txt", "w") as file:
    for item in rag_list:
        file.write(f"{item}\n")

100%|██████████| 1637/1637 [00:00<00:00, 10264.94it/s]


## Generate dev set

In [3]:
# load excel files in dev set folder
import os

dev_set_folder = "devset"

dfs = []
for file in os.listdir(dev_set_folder):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(dev_set_folder, file))
        dfs.append(df)

# merge all excel
dev_set = pd.concat(dfs, ignore_index=True)

# add csv
rag_batch_1_with_qa = pd.read_csv("devset/rag_batch_1_with_qa.csv", sep=";").iloc[:, 1:].dropna()
rag_batch_1_with_qa.columns = dev_set.columns
dev_set = pd.concat([dev_set, rag_batch_1_with_qa], ignore_index=True)

dev_set

,"question, str","answer, str","text, str","pnumber, str","law number, str"
0,"Hvad har ejeren af en ejerlejlighed, sammen me...","Grunden, fælles bestanddele og tilbehør",'Ejeren af en ejerlejlighed har sammen med and...,3,LOV nr 908 af 18/06/2020
1,Hvem fastsætter eller aftaler bestemmelser om ...,Finansministeren fastsætter eller aftaler best...,'Højskolen skal følge de af finansministeren f...,30,LBK nr 780 af 08/08/2019
2,Hvad skal Beskæftigelsesministeriet og Finanst...,Den indsendte årsrapport skal i det mindste in...,'Uden ugrundet ophold efter repræsentantskabet...,25 l,LBK nr 1110 af 10/10/2014
3,Hvor mange procent må kapitalandele i og lån y...,Kapitalandele i og lån ydet til en virksomhed ...,'Følgende grænser for Arbejdsmarkedets Tillægs...,26 e,LBK nr 1110 af 10/10/2014
4,Hvad er en betingelse for retten til jobpræmie?,Det er en betingelse for retten til jobpræmie ...,'Det er en betingelse for retten til jobpræmie...,9,LOV nr 287 af 29/03/2017
...,...,...,...,...,...
101,Hvordan anføres kandidatlister på stemmesedler?,I særskilte felter.,Kandidatlisterne anføres på stemmesedlen i sær...,46,LBK nr 6 af 08/01/2024
102,Hvem iværksætter beslaglæggelse?,Politiet.,Politiet iværksætter beslaglæggelse. Politiet ...,807,LBK nr 250 af 04/03/2024
103,Hvis interesser skal foranstaltninger mod inte...,De forvaltede alternative investeringsfondes e...,En forvalter af alternative investeringsfonde ...,23,LBK nr 231 af 01/03/2024
104,Hvad skal valgstyrere eller tilforordnede vælg...,At stemmekasserne er tomme.,Afstemningen begynder kl. Inden stemmeafgivnin...,38,LBK nr 1432 af 01/12/2023


## Vectorize retrieval corpus

### Sparse retrieval

In [4]:
rag_list2 = rag_list

def preprocess(rag_list):
    # extract and preprocess text
    corpus = [item['text'] for item in rag_list]
    corpus = [re.sub('\\s{2,}', ' ', 
                     re.sub('\\W|[0-9]|§', ' ',
                           item.lower())) for item in corpus]

    # remove stopwords
    #nltk.download('punkt')
    stop_words = set(stopwords.words('danish'))
    corpus = [' '.join(word for word in text.split() 
                      if word not in stop_words) for text in tqdm(corpus)]
    
    return corpus

corpus = preprocess(rag_list2)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

100%|██████████| 42593/42593 [00:00<00:00, 117957.46it/s]


### Dense retrieval

In [5]:
## WRITE LATER

## RAG retriever

### Sparse retrieval pipeline


In [148]:
def sparse_retrieval(question, sparse_matrix, k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """

    # preprocess and vectorize question
    question_processed = [re.sub('\\s{2,}', ' ', 
                               re.sub('\\W|[0-9]|§', ' ',
                                     question.lower()))]
    
    # remove stopwords
    stop_words = set(stopwords.words('danish'))
    question_processed = [' '.join(word for word in text.split() 
                                 if word not in stop_words) for text in question_processed]
    
    question_vector = vectorizer.transform(question_processed)

    # sparse retrieval (cosine similarity)
    sparse_retrieval = X.dot(question_vector.T).toarray()

    # get top k paragraphs
    top_k = np.argsort(sparse_retrieval.flatten())[-k:]

    return top_k

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
top_k = sparse_retrieval(random_question, X)
for i in top_k:
    print(f'{rag_list2[i]["paragraf_nr"]}: {rag_list2[i]["text"]}')

Hvor længe kan der gives tilladelse til afholdelse af lokale puljevæddemål på cykelløb på bane, hundevæddeløb på væddeløbsbane, kapflyvning med duer og hestevæddeløb? 

§ 25.: Tilladelse til udbud og arrangering af spil kan gives til personer og selskaber m.v. (juridiske personer), medmindre andet fremgår af denne lov, jf. dog stk. 2. Tilladelse til udbud og arrangering af lokale puljevæddemål på cykelløb på bane, hundevæddeløb på væddeløbsbane og kapflyvning med duer, jf. § 13, kan kun gives til selskaber m.v. (juridiske personer), der er arrangør af cykelløb på bane, hundevæddeløb på væddeløbsbaner eller kapflyvning med duer, og som er tilsluttet den pågældende sportsgrens centralorganisation eller -forbund.
§ 13.: Der kan gives tilladelse til afholdelse af lokale puljevæddemål på cykelløb på bane, hundevæddeløb på væddeløbsbane, kapflyvning med duer og hestevæddeløb. Tilladelse kan gives for indtil 3 år og for et bestemt antal dage for de enkelte år. Når begivenheden, der indgås væd

### Create embedding corpus

In [7]:
# creating embedding corpus with the 'KennethTM/bert-base-uncased-danish'

# bert_tokenizer = AutoTokenizer.from_pretrained("KennethTM/bert-base-uncased-danish")
# bert_model = AutoModel.from_pretrained("KennethTM/bert-base-uncased-danish")

# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

# cls_embeddings = []

# idx = 0

# for item in tqdm(rag_list):
#     # doing a try and except as some paragraphs may exceed the context window of the BERT (I believe)
#     try:
#         # tokenize texts
#         input_ids = bert_tokenizer.encode(item['text'], return_tensors='pt')
#         # run through BERT
#         with torch.no_grad():  # disable gradient computation for inference
#             outputs = bert_model(input_ids)
#         # extract cls-token
#         cls_vector = outputs.last_hidden_state[:, 0, :]
#         # add cls-vector to list of embeddings
#         cls_embeddings.append(cls_vector)
#     except:
#         # if error then count errors with this
#         idx += 1

# print(f'{idx} no. of errors')

# # concatenate list into torch tensor
# cls_embeddings_tensor = torch.cat(cls_embeddings, dim=0)

In [30]:
# saving the tensor
#torch.save(cls_embeddings_tensor, '/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_tensor.pt')

In [9]:
# loading the tensor

# cls_embeddings_tensor = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_tensor.pt')

In [12]:
# # creating embedding corpus with the 'vesteinn/DanskBERT'

# tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")
# model = AutoModel.from_pretrained("vesteinn/DanskBERT")

# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

# cls_embeddings = []

# idx = 0

# for item in tqdm(rag_list):
#     # doing a try and except as some paragraphs may exceed the context window of the BERT (I believe)
#     try:
#         # tokenize texts
#         input_ids = tokenizer.encode(item['text'], return_tensors='pt')
#         # run through BERT
#         with torch.no_grad():  # disable gradient computation for inference
#             outputs = model(input_ids)
#         # extract cls-token
#         cls_vector = outputs.last_hidden_state[:, 0, :]
#         # add cls-vector to list of embeddings
#         cls_embeddings.append(cls_vector)
#     except:
#         # if error then count errors with this
#         idx += 1

# print(f'{idx} no. of errors')

# # concatenate list into torch tensor
# cls_embeddings_tensor_DanskBERT = torch.cat(cls_embeddings, dim=0)

In [ ]:
# saving the DanskBERT-tensor
#torch.save(cls_embeddings_tensor_DanskBERT, '/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_DanskBERT.pt')

In [ ]:
# loading the DanskBERT-tensor
#cls_embeddings_tensor_DanskBERT = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_DanskBERT.pt')

### Dense retrieval pipeline

In [17]:
bert_tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")
bert_model = AutoModel.from_pretrained("vesteinn/DanskBERT")

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at vesteinn/DanskBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [150]:
def dense_retrieval(question, k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """
    
    # Encode the input sentence
    input_ids = bert_tokenizer.encode(question, return_tensors="pt")  # Encode and add batch dimension
    # Pass the input through the model
    
    with torch.no_grad():  # disable gradient computation for inference
        outputs = bert_model(input_ids)

    # Extract the CLS token representation
    cls_vector = outputs.last_hidden_state[:, 0, :]  # CLS token is at position 0
    
    # sparse retrieval (cosine similarity)
    dense_retrieval = cls_embeddings_tensor_DanskBERT @ torch.transpose(cls_vector, 0, 1)
    
    # get top k paragraphs
    top_k_indices = torch.sort(dense_retrieval, descending=True, dim=0)[1][:k]

    return top_k_indices

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
top_k = dense_retrieval(random_question, k=3)
for i in top_k:
    print(f'{rag_list[i]["text"]}')

Bedriftværn, der er etableret i henhold til den lovgivning, der er gældende indtil 1. januar 1993, opretholdes, medmindre at hvad? 

Inden der indgås aftale om køb af en pakkerejse, skal den rejsende gives følgende oplysninger, hvis det er relevant for pakkerejsen, herunder om rejseydelsernes væsentligste kendetegn: Rejsedestination, rejserute, varigheden af og tidspunktet for opholdet og, hvis indkvartering er inkluderet, antallet af inkluderede overnatninger, Rejsedestination, rejserute, varigheden af og tidspunktet for opholdet og, hvis indkvartering er inkluderet, antallet af inkluderede overnatninger, anvendte befordringsmidler og deres kendetegn og kategori, sted, dato og tidspunkt for af- og hjemrejse og oplysninger om steder, hvor der gøres ophold undervejs, med angivelse af disse opholds varighed og transportforbindelser, indkvarteringsstedets beliggenhed, væsentligste kendetegn og kategori i henhold til destinationslandets regler, inkluderede måltider, besøg, udflugter eller 

In [ ]:
# fejl i 6, mangler spørgsmålstegn
# 7 er et dårligt spørgsmål
# 21 er også lidt dårlig
# 35 er ukomplet
# 40 og 41 er de samme spørgsmål

